In [ ]:
import geopandas
import pandas
import pathlib
import numpy
import matplotlib
import matplotlib.pyplot
from IPython.display import display, HTML

display(HTML("<style>.container { width:100% !important; }</style>"))

# Plan
* Load in GeoFabrics estimated width, depth and water surface height - (using those used in GeoFabrics - i.e. same level of smoothing)
  * Calculate channel area - assume rectangular
* Load in measured cross sections
  * Transform to NZVD2016
  * Calculate the channel area - use the same water height as the GeoFabrics estimates
* Compare the results
  * Take the nearest GeoFabrics cross section to each measured cross section
  * Plot area's on the same plot

In [ ]:
sampling_resolution = 10
threshold = 1

### Load in GeoFabrics Bathymetry estimates

In [ ]:
cache_path = pathlib.Path(fr"C:\Users\pearsonra\Documents\data\river_bathemetry\waikanae\caches\100000000_new")
widths = geopandas.read_file(cache_path / "river_characteristics.geojson")
depths = geopandas.read_file(cache_path / "river_bathymetry.geojson")
main_channel = geopandas.read_file(cache_path / "rec_channel_100000000.geojson")

### Specify GeoFabric parameter
Both the column names used in calculations, and also select parameter file values

In [ ]:
water_elevation_name = 'min_z_centre_unimodal'
full_width_name = 'widths_mean_0.25km'
flat_width_name = 'flat_widths_mean_0.25km'
threshold_name = 'thresholds_mean_0.25km'
rupp_bed_elevation_name = 'bed_elevation_Rupp_and_Smart'
neal_bed_elevation_name = 'bed_elevation_Neal_et_al'
min_bank_height= 0.75
rupp_depth_name = 'bed_depth_Rupp_and_smart'
neal_depth_name = 'bed_depth_Neal_et_al'

#### Calculate depths from bed elevation

In [ ]:
depths[rupp_depth_name] = widths[water_elevation_name]-depths[rupp_bed_elevation_name]
depths[neal_depth_name] = widths[water_elevation_name]-depths[neal_bed_elevation_name]

#### Convert back to full bankfill area
Function to do the conversion

In [ ]:
def to_full_bank_flow_area(depth_dataframe, width_dataframe, min_bank_height, threshold_name, full_width_name, flat_width_name, depth_name, water_elevation_name):
    
    # backwards calculate the 'flat water area' - note this is in error as didn't remove most of threshold area
    over_estimated_flat_water_area = depth_dataframe[depth_name] * width_dataframe[flat_width_name]

    # Add/correct for area along the bank edges (approximate as a triangle - two triangles = a rectange)
    displaced_bank_edge_area = (width_dataframe[full_width_name] - width_dataframe[flat_width_name]) \
        * (width_dataframe[threshold_name] - min_bank_height)
    
    # Extra flood water in threshold
    extra_flood_water_area = (width_dataframe[threshold_name] - min_bank_height) * width_dataframe[flat_width_name]

    # Calculate the area estimated for full bank width flow
    full_bank_area = over_estimated_flat_water_area + displaced_bank_edge_area
    flat_water_area = full_bank_area - extra_flood_water_area

    return flat_water_area, full_bank_area
    

### Plot full bank flow area and flat water area
For Rupp and Smart

In [ ]:
flat_water_area, full_bank_area = to_full_bank_flow_area(depth_dataframe=depths,
                                                         width_dataframe=widths,
                                                         min_bank_height=min_bank_height,
                                                         threshold_name=threshold_name,
                                                         full_width_name=full_width_name,
                                                         flat_width_name=flat_width_name,
                                                         depth_name=rupp_depth_name,
                                                         water_elevation_name=water_elevation_name)
areas = pandas.concat([flat_water_area, full_bank_area], keys=['flat_water_area', 'full_bank_area'], axis=1)

In [ ]:
areas.plot()

## Load in Measured cross sections
load in files including a raster for performing a local dataum conversion

In [ ]:
cross_section_file = pathlib.Path(r"C:/Users/pearsonra/Documents/data/river_bathemetry/sections/Waikanae/Waikanae_Cross_Sections_NIWA/2014_Survey_XS_NZTM.shp")
cross_sections = geopandas.read_file(cross_section_file)
local_datum_file = pathlib.Path(r"C:\Users\pearsonra\Documents\data\local_datums\lds-wellington-1953-to-nzvd2016-conversion-raster-GTiff"
                                "\wellington-1953-to-nzvd2016-conversion-raster.tif")
import rioxarray
local_vertical_datum = rioxarray.rioxarray.open_rasterio(local_datum_file, masked=True)

### Perform LVD to NZVD2016 conversion

In [ ]:
def get_lvd_conversion(geometry, local_vertical_datum):
    return local_vertical_datum.sel({'x': [geometry.x], 'y': [geometry.y]}, method='nearest').data[0,0,0]

cross_sections['lvd_conversion'] = cross_sections.apply(lambda row: get_lvd_conversion(row.geometry, local_vertical_datum), axis=1)
cross_sections['z_2016'] = cross_sections['Level'] - cross_sections['lvd_conversion']

### Get nearest estimated to measured cross section
Make cross sections geometries

In [ ]:
section_centres = {'Section': [], 'geometry': []}
for section in cross_sections['Section'].unique():
    cross_section = cross_sections[cross_sections['Section'] == section]
    section_centre = cross_section[cross_section['Level']==cross_section['Level'].min()].iloc[0]
    section_centres['Section'].append(section_centre['Section'])
    section_centres['geometry'].append(section_centre['geometry'])
section_centres = geopandas.GeoDataFrame(section_centres, crs=main_channel.crs)

Cycle through each section identifying the nearest section in the key parameters

In [ ]:
nearest_cross_sections = numpy.zeros_like(widths[water_elevation_name], dtype=bool)
nearest_sections_ids = numpy.zeros_like(widths[water_elevation_name])
for index, row in section_centres.iterrows():
    nearest_cross_sections[widths.distance(row.geometry).argmin()] = True
    nearest_sections_ids[widths.distance(row.geometry).argmin()] = row['Section']

nearest_widths = widths[nearest_cross_sections]
nearest_depths = depths[nearest_cross_sections]
nearest_areas = areas[nearest_cross_sections]
nearest_widths.insert(loc=0, column='Section', value=nearest_sections_ids[nearest_cross_sections])
nearest_depths.insert(loc=0, column='Section', value=nearest_sections_ids[nearest_cross_sections])

Calculate the measured 'depth' based on the water surface in the LiDAR 

In [ ]:
cross_sections[water_elevation_name] = cross_sections.apply(lambda row: nearest_widths[nearest_widths['Section'] == row['Section']][water_elevation_name].mean(), axis=1)
cross_sections['depth'] = cross_sections[water_elevation_name] - cross_sections['z_2016']

### Calculate cross sectional area of each profile
From flat water height, and also the bank-full height. Cycle through sections taking each section.

_Could add a check to ensure no gap - i.e. portion of the channel above water then below again - but won't as could be two channels_

#### Initial estimate 
Calculate the Riemann squares centred sections entirely less than water surface. 

In [ ]:
measured_area = []
for index, row in nearest_widths.iterrows():
    section = cross_sections[cross_sections['Section'] == row['Section']]
    section = section[section['depth'] > 0]
    measured_area.append((section['Offset'].rolling(window=2).apply(numpy.diff)
      * (section['depth']).rolling(window=2).apply(numpy.mean)).iloc[1:].sum())
nearest_areas.insert(loc=0, column='Measured areas crude', value=measured_area)

#### Calculated propely
Calculate area from where each section goes below the water surface (even if between two measurement sites)

In [ ]:
def zero_crossing(x_pos: float, x_neg: float, y_pos: float, y_neg: float):
    x_zc = x_pos * -y_neg / (y_pos - y_neg) + x_neg * y_pos / (y_pos - y_neg)
    return x_zc

In [ ]:
measured_area = []

for index, row in nearest_widths.iterrows():
    section = cross_sections[cross_sections['Section'] == row['Section']]
    section = section.reset_index(drop=True)
    water_elevation = row[water_elevation_name]
    area = 0
    for i in range(len(section) - 1):
        if section['depth'][i] >= 0 and section['depth'][i + 1] >= 0:
            # calculate Raimann square area
            area += (section['Offset'][i + 1] - section['Offset'][i]) * (section['depth'][i] + section['depth'][i + 1]) / 2
        elif section['depth'][i] > 0:
            # Calculate from the left - right is above
            offset_crossing = zero_crossing(x_pos=section['Offset'][i + 1], x_neg=section['Offset'][i],
                                            y_pos=section['depth'][i + 1], y_neg=section['depth'][i]) 
            area += (offset_crossing - section['Offset'][i]) * section['depth'][i] / 2
        elif section['depth'][i + 1] > 0:
            # Calculate from the right - left is above
            offset_crossing = zero_crossing(x_pos=section['Offset'][i], x_neg=section['Offset'][i + 1],
                                            y_pos=section['depth'][i], y_neg=section['depth'][i + 1]) 
            area += (section['Offset'][i + 1] - offset_crossing) * section['depth'][i + 1] / 2
    measured_area.append(area)
nearest_areas.insert(loc=0, column='Measured areas exact', value=measured_area)

In [ ]:
f, axs = matplotlib.pyplot.subplots(1, 2, sharey=True, figsize=(15,6))
nearest_areas.plot(ax=axs[0])
areas.plot(ax=axs[1])

## Plot bed elevations and depths of the two approaches

In [ ]:
f, axs = matplotlib.pyplot.subplots(1, 2, figsize=(15, 6))

nearest_widths.set_index('Section', drop=True)[water_elevation_name].plot(ax=axs[0], label='Water surface elevation')
nearest_depths.set_index('Section', drop=True)['bed_elevation_Neal_et_al'].plot(ax=axs[0], label='Bed elevation calculated Neal et al')
nearest_depths.set_index('Section', drop=True)['bed_elevation_Rupp_and_Smart'].plot(ax=axs[0], label='Bed elevation calculated Rupp & Smart')
cross_sections.groupby('Section')['z_2016'].min().plot(ax=axs[0], label='Measured cross-section')
axs[0].set(title="Bed elevations estimated and measured", xlabel="Transect number where there are measured cross sections", ylabel="River bed elevation in m")
axs[0].legend()

nearest_depths.set_index('Section', drop=True)[neal_depth_name].plot(ax=axs[1], label='Calculated Neal et al depth')
nearest_depths.set_index('Section', drop=True)[rupp_depth_name].plot(ax=axs[1], label='Calculated Rupp & Smart depth')
cross_sections.groupby('Section')['depth'].max().plot(ax=axs[1], label='Measured cross-section')
axs[1].set(title="Bed depths estimated and measured", xlabel="Transect number where there are measured cross sections", ylabel="River depths in m")
axs[1].legend()

## Plot cross sections

In [ ]:
cross_sections[[water_elevation_name, 'z_2016']].plot(figsize=(15, 6))
matplotlib.pyplot.yticks(ticks=[-2.5, 0, 2.5, 10, 20, 30, 40]);
matplotlib.pyplot.grid(axis="y")

In [ ]:
f, axs = matplotlib.pyplot.subplots(2, 4, figsize=(30, 12), sharey=True)
for key, group in cross_sections.groupby('Section'):
    (-group['depth']).reset_index(drop=True).plot(ax=axs[int(key/10/28), int((key/10 - 28)/7)], label=f"{key}")
for axi in axs:
    for axij in axi:
        axij.legend()

In [ ]:
f, axs = matplotlib.pyplot.subplots(2, 4, figsize=(30, 12), sharey=True)

for key, group in cross_sections.groupby('Section'):
    group = group.copy(deep=True)
    group.loc[group['depth'] < 0, 'depth'] = numpy.nan
    (-group['depth']).reset_index(drop=True).plot(ax=axs[int(key/10/28), int((key/10 - 28)/7)], label=f"{key}")
    #print(f"key={key}, f{group['depth'].values}")
for axi in axs:
    for axij in axi:
        axij.legend()

In [ ]:
cross_sections.groupby('Section')['depth'].plot();

In [ ]:
cross_sections.groupby('Section')['z_2016'].plot();

### Plots exploring the change to width to produce an expected bed elevation

In [ ]:
fudge_factor = 2
depth_wider = (nearest_key_parameters['mannings_n'] * nearest_key_parameters['flow'] / (numpy.sqrt(nearest_key_parameters['slope']) * nearest_key_parameters['widths_Savgol'] * fudge_factor)) ** (3/5) - threshold

f, ax = matplotlib.pyplot.subplots(figsize=(20, 6))
(nearest_key_parameters['min_z_centre_unimodal']-depth_wider).reset_index(drop=True).plot(label='Calculated from unimodal min_z')
(nearest_key_parameters['min_z_centre_unimodal_0.5km_rolling_mean'] - depth_wider).reset_index(drop=True).plot(label='Calculated from smooth unimodal min_z')
(nearest_key_parameters['min_z_centre']-depth_wider).reset_index(drop=True).plot(label='Calculated from min_z')
cross_sections.groupby('Section')['z_2016'].min().reset_index(drop=True).plot(label='Measured cross-section')
ax.set(title=f"Depths estimated using the Uniform Flow Throery approach with width mutiplier of {fudge_factor}")
matplotlib.pyplot.xlabel(f"Transect number (spaced every {sampling_resolution}m upstream)")
matplotlib.pyplot.ylabel(f"River depth in m");
matplotlib.pyplot.legend()